# Reporte Solicitudes Ara

In [ ]:
import os

from_drive = True  # same flag you use everywhere

if os.environ.get("ATLAS_BOOTSTRAPPED") != "1":
    # ---------- GIT ON COLAB ONLY ----------
    try:
        from google.colab import userdata

        git_token = userdata.get('gitToken')
        git_user = userdata.get('gitUser')
        git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
        branch_to_pull = 'dev'

        os.chdir('/content')

        if not os.path.isdir('Atlas'):
            !git clone {git_url}

        %cd Atlas
        !git fetch origin {branch_to_pull}
        !git checkout {branch_to_pull}
        !git pull origin {branch_to_pull}

        !pip install -r PipelinesConsumo/src/requirements.txt
        %cd PipelinesConsumo

    except Exception as e:
        print(e)
        print('Running in other environment not colab probably!')

    # ---------- DRIVE + SHEETS ----------
    if from_drive:
        from pydrive2.auth import GoogleAuth
        from pydrive2.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe
        import gdown

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        creds, _ = default()
        gc = gspread.authorize(creds)

    os.environ["ATLAS_BOOTSTRAPPED"] = "1"
else:
    print("Bootstrap already done, assuming orchestrator ran it.")

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns,
                       remove_accents)
from PipelinesConsumo.src.rawAtlas import RawAtlas
from PipelinesConsumo.src.processedAtlas import ProcessedAtlas
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_sheets_in_drive_folder,
                             update_sheets_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder,
                             create_csv_file_in_drive_folder,
                             write_csv_to_drive,
                             read_csv_from_drive)
from src.constants import (atlas_raw_output_folder_id,
                           atlas_consumo_output_folder_id,
                           consumo_sheets_ids_dict,
                           data_source_folder_id,
                           raw_output_ids,
                           folder_id_bauto_gabo,
                           id_reporte_ventas,
                           id_edas_referenciados,
                           id_torre_de_control
                           )


warnings.filterwarnings('ignore')



## Load

In [ ]:
folder_id_catalogos = "1TE24Yl4lQ6ZxHJSw_ZDzb6W4XpB-21FL"
id_catalogos_status = list_file_ids_for_drive_folder(drive,folder_id_catalogos)['CatalogoSolicitudesAprobacion']
cat_tarea_actual = (read_from_google_sheets(gc,id_catalogos_status,sheetname='CatTareaActual')
                    .assign(tareaactual = lambda x: x.nb_tarea_actual.apply(remove_accents).str.strip().str.upper(),
                            status_automarket = lambda x: x.status_am.apply(remove_accents).str.strip().str.upper())
                    .drop(columns=['nb_tarea_actual','status_am'])
                    )
cat_decision_sistema = (read_from_google_sheets(gc,id_catalogos_status,sheetname='CatDecisionSistema')
                      .assign(decisionsistema = lambda x: x.decisionsistema.apply(remove_accents).str.strip().str.upper(),
                              status_riesgos = lambda x: x.riesgos.apply(remove_accents).str.strip().str.upper())
                      .drop(columns=['riesgos'])
                        )
# edas, gabo, torre de control
edas = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcEdas'])
bauto = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcConsolidadoBautoLastStatus'])
torreRaw = read_from_google_sheets(gc,id_torre_de_control,sheetname='Asignación compradores')
torreRaw.columns
clientes = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcClientes'])
# clientes
subset_columns = ['id_lead','origen_automarket','id_comprador','id_de_ultimo_pedido','folio_bauto',
                  'espacio_automarket','asesor_de_ventas','fecha_de_asignacion','total_apartados',
                  'estatus_de_lead','lead_contactado_(visualiza_cc)',
                  'fecha_de_cierre_del_lead','motivos_de_cancelacion',
                  'correo_recibido_en_buzon_contingencia','documentacion_completa_contingencia',
                  ]
rename_dict = {'lead_contactado_(visualiza_cc)':'lead_contactado'}
torre = (torreRaw
.pipe(process_columns)
[subset_columns]
 .assign(folio = lambda x: pd.to_numeric(x['folio_bauto'],errors='coerce').astype('Int64'),
         id_comprador = lambda x: pd.to_numeric(x['id_comprador'],errors='coerce').astype('Int64'),
         fecha_de_asignacion = lambda x: pd.to_datetime(x['fecha_de_asignacion'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
        #  fecha_de_cierre_del_lead = lambda x: pd.to_datetime(x['fecha_de_cierre_del_lead'],format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
         )
 .rename(columns=rename_dict)
 .sort_values(by='fecha_de_asignacion',ascending=False)
 )
folder_id_folios_rod = '1OW4yxE7h8BCcn0mqhCfk05B4_27Ghvfd'
files_rod = list_file_ids_for_drive_folder(drive,folder_id_folios_rod)
files = list(files_rod.keys())
latest_rod_id = files_rod.get(files[0]) ## cambiar este siempre
print(files[0])
latest_rod_id
from_drive_to_local(drive,latest_rod_id,'rod_latest.xlsx')
rod = (pd.read_excel('rod_latest.xlsx')
      .rename(columns={'Name':'intento',
                       'MX_ATN_Id_Simulacion__c':'n_simulacion',
                       'MX_ATN_creditId__c':'folio',
                       'MX_ATN_Account__r.Name':'name',
                       'MX_ATN_Account__r.MX_ATN_CommerceId__c':'id_am',
                       'MX_ATN_Account__r.MX_ATN_PrimaryContact__r.Email':'email',
                       'MX_ATN_Account__r.MX_ATN_PrimaryContact__r.MobilePhone':'phone',
                       'MX_ATN_Status__c':'status'})
      .assign(CreatedDate = lambda x: pd.to_datetime(x.CreatedDate, format="%d/%m/%Y, %H:%M"),
              phone = lambda x: x.phone.astype('Int64').astype(str))
      )

## Transform

### folios rod by date


In [ ]:
bauto_mod_api = (bauto
             .assign(flag_eda = lambda x: np.where(x.folio.isin(edas.folio.unique()),1,0),
                     flag_tc = lambda x: np.where(x.folio.isin(torre.folio.unique()),1,0),
                     origen_real= lambda x: np.where(x.folio.isin(edas.folio.unique()),'EDA',x.origen),
                     decisionsistema = lambda x: x.decisionsistema.apply(remove_accents).str.strip().str.upper(),
                     tareaactual = lambda x: x.tareaactual.apply(remove_accents).str.strip().str.upper(),
                     telefono = lambda x: x.telefono.astype('Int64')
                     )
             [lambda x: x.origen_real=='API']
             .merge(cat_tarea_actual,on='tareaactual',how='left')
             .merge(cat_decision_sistema,on='decisionsistema',how='left')
             .drop_duplicates()
             .sort_values(by='fecha_creacion',ascending=False)
             )


dummies_status_df = pd.get_dummies(bauto_mod_api.status_automarket)*1
dummies_status_df.columns = [f'status_automarket_{x}' for x in dummies_status_df.columns]
bauto_mod_api = pd.concat([bauto_mod_api,dummies_status_df],axis=1)


bauto_by_date=(bauto_mod_api
 .groupby('fecha_creacion',as_index=False)
 .agg(bauto_folios_api=('folio','nunique'),
      bauto_folios_en_proceso = ('status_automarket_PROCESO DESEMBOLSO','sum'),
      bauto_folios_desembolso = ('status_automarket_DESEMBOLSADO','sum'),
      bauto_folios_rechazado = ('status_automarket_RECHAZADO','sum'),
      bauto_folios_inicio_solicitud = ('status_automarket_INICIO SOLICITUD','sum'),
      bauto_folios_revision_riesgo = ('status_automarket_REVISION RIESGOS','sum'),
     
      )
 .rename(columns={'fecha_creacion':'date'})

 )

### folios rod by date

In [ ]:
rod_by_date = (rod
 .assign(date = lambda x:pd.to_datetime(x.CreatedDate).dt.strftime('%Y-%m-%d'),
         id_am_con_folio = lambda x: np.where(x.folio.notna(),x.id_am,np.nan))
.groupby('date',as_index=False)
.agg(fapi_intentos_solicitud=('intento','nunique'),
     fapi_usuarios_con_intento=('id_am','nunique'),
     fapi_folios=('folio','nunique'),
     fapi_usuarios_con_folio=('id_am_con_folio','nunique'),
      )
 )